In [24]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 

from skimage.transform import resize

In [60]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename) 
    
    # Check Modality
    if (ds.Modality != 'DX'):
        print('Invalid Modality to analyze.')
        return None

    # Check Modality
    if (ds.BodyPartExamined != 'CHEST'):
        print('Invalid Body Part Examined to analyze.')
        return None

    # Check Modality
    #print(ds.PatientPosition)
    if (ds.PatientPosition != 'PA') and (ds.PatientPosition != 'AP'):
        print('Invalid Patient Position to analyze.')
        return None
        
    img = ds.pixel_array
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 

    proc_img = img.copy()

    #print('copy made')
    #print(proc_img)
    
    # Finally resizing it to our desired size.
    proc_img = resize(proc_img, img_size)
    
    proc_img = proc_img / 255.0

    #proc_img = (proc_img - img_mean)/img_std
   
    return proc_img


# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):

    with open("my_model.json", "r") as json_file:
        model_json = json_file.read()
    
    model = keras.models.model_from_json(model_json)
    model.load_weights(weight_path)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    prediction = 'No Pneumonia Finding'    
    
    #print(img.shape)
    #print(img)
    
    pred = model.predict(img)
    
    print(pred)
    print(pred[0])
    if pred > thresh:
        prediction = 'Pneumonia Finding'        
    
    return prediction 

In [ ]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'                                 #path to saved model
weight_path = "{}_my_model.best.hdf5".format('xray_class')    #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
#img_mean =  127.0759867073764 # loads the mean image value they used during training preprocessing
#img_std = 43.14987587779298  # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path)
thresh = 0.5125144      #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    #print(img)
    if img is None:
        continue

    img_mean = np.mean(img)
    img_std = np.std(img)
    
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)